In [27]:
# this is after (GO) 2023-02-08

# Load DF

In [28]:
import pandas as pd
import numpy as np
import config

In [29]:
# specify number of member

df_count = pd.read_excel(config.path_member_count, sheet_name="Data - After Migration")
df_count.columns = df_count.columns.str.lower().str.replace(" ", "_")
df_count["year"] = df_count["month"].dt.year
df_count["month"] = df_count["month"].dt.month

# ! if exclude cpt, use number without cpt
date_mask = (df_count["month"] == config.month_num) & (df_count["year"] == config.year)
if config.exclude_cpt:
    num_member = df_count.loc[date_mask, "total_member_without_cpt"].values[0]
    num_member_dlx = df_count.loc[date_mask, "number_member_del_without_cpt"].values[0]
    num_member_vip = df_count.loc[date_mask, "number_member_vip_without_cpt"].values[0]
    num_member_go = df_count.loc[date_mask, "number_member_go_without_cpt"].values[0]
else:
    num_member = df_count.loc[date_mask, "total_member"].values[0]
    num_member_dlx = df_count.loc[date_mask, "number_member_del"].values[0]
    num_member_vip = df_count.loc[date_mask, "number_member_vip"].values[0]
    num_member_go = df_count.loc[date_mask, "number_member_go"].values[0]

df_month = df_count.loc[date_mask, "month"].values[0]
print(f"Month: {df_month}")
print(f"Total member: {num_member}")
print(f"Deluxe member: {num_member_dlx}")
print(f"VIP member: {num_member_vip}")
print(f"GO member: {num_member_go}")

Month: 12
Total member: 4016.0
Deluxe member: 2474.0
VIP member: 258.0
GO member: 1284.0


In [30]:
# load and clean df att
df_att_ori = pd.read_excel(config.path_att)
df_att = df_att_ori.copy()
df_att = (df_att
    # exclude unattendance
    .loc[df_att["student_attendance"] == "Attend"]  
    # exclude GOC for vip
    .loc[lambda df_:  ~((df_["class_type"] == "GOC") & (df_["student_membership"] == "VIP"))]
)
# note: if exclude cpt, take out cpt student
if config.exclude_cpt:
    df_att = df_att.loc[~(df_att["student_code"].str.lower().str.contains("cpt"))]

In [31]:
# load and clean df session
path_sess = str(config.path_att).replace("data attendance", "data session")
df_sess = pd.read_excel(path_sess)
# note: exclude GOC for vip
df_sess = df_sess.loc[
    ~((df_sess["class_type_grouped"] == "GOC") & (df_sess["class_service"] == "VIP"))
]

# Calculation

In [32]:
# level 1: go vip del

perc_go = round(((num_member_go / num_member) * 100), 0)
perc_dlx = round(((num_member_dlx / num_member) * 100), 0)
perc_vip = round(((num_member_vip / num_member) * 100), 0)

In [33]:
# level 2: active inactive

# vip
num_vip_ac = df_att.loc[
    (df_att["student_membership"] == "VIP"), "student_code"
].nunique()
num_vip_in = num_member_vip - num_vip_ac
perc_vip_ac = round(((num_vip_ac / num_member_vip) * 100), 0)
perc_vip_in = round(((num_vip_in / num_member_vip) * 100), 0)

# deluxe
num_dlx_ac = df_att.loc[
    (df_att["student_membership"] == "Deluxe"), "student_code"
].nunique()
num_dlx_in = num_member_dlx - num_dlx_ac
perc_dlx_ac = round(((num_dlx_ac / num_member_dlx) * 100), 0)
perc_dlx_in = round(((num_dlx_in / num_member_dlx) * 100), 0)

# go
num_go_ac = df_att.loc[(df_att["student_membership"] == "GO"), "student_code"].nunique()
num_go_in = num_member_go - num_go_ac
perc_go_ac = round(((num_go_ac / num_member_go) * 100), 0)
perc_go_in = round(((num_go_in / num_member_go) * 100), 0)

In [34]:
# level 3: eligible for online
# starting from july, no need to find num_online
num_onl = num_dlx_ac + num_go_ac

In [35]:
# level 4: vip one on one vs vpg

df_vip = (
    df_sess.loc[df_sess["class_service"] == "VIP"]
    .assign(
        class_type_grouped=lambda df_: df_["class_type_grouped"]
        .str.replace("Offline ", "")
        .str.replace("Online ", "")
    )
    .groupby("class_type_grouped")
    .agg(count=("class_type_grouped", "size"))
)

In [36]:
def catchstate(df, var_name: str) -> "pd.DataFrame":
    """
    Helper function that captures intermediate Dataframes mid-chain.
    In the global namespace, make a new variable called var_name and set it to dataframe
    """
    globals()[var_name] = df
    return df

In [37]:
# level 5: basket for online
# ! GO

# online
df_onl_go = (df_att
    .loc[(df_att['student_membership'] == 'GO') &
        (df_att['class_mode'].isin(['Online', 'GOC'])) &
        (df_att['class_type'].isin(['Online Encounter','Online Complementary', 'Online Social Club'])) 
    ]
    .groupby(['class_type', 'student_code'])
    .agg(
        count= ('student_code', 'size')
    )
    .reset_index()
)


# enc
bins = [0, 1, 3, 1_000_000]
df_enc_go = (df_onl_go
    .loc[df_onl_go['class_type'] == 'Online Encounter']
    .groupby(['class_type', pd.cut(df_onl_go['count'], bins, right= False)])
    .size()
    .to_frame()
    .rename({0: 'count'}, axis= 'columns')
    .pipe(catchstate, var_name= "df_catched") # catch the current state of df at this point
    .replace(
        to_replace= 0,
        value= num_go_ac - df_catched['count'].sum()
    )
    .assign(
        perc= lambda x: round(((x['count'] / x['count'].sum()) * 100), 0)
    )
)

# complementary
bins = [0, 1, 4, 1_000_000]
df_cc_go = (df_onl_go
    .loc[df_onl_go['class_type'] == 'Online Complementary']
    .groupby(['class_type', pd.cut(df_onl_go['count'], bins, right= False)])
    .size()
    .to_frame()
    .rename({0: 'count'}, axis= 'columns')
    .pipe(catchstate, var_name= "df_catched") # catch the current state of df at this point
    .replace(
        to_replace= 0,
        value= num_go_ac - df_catched['count'].sum()
    )
    .assign(
        perc= lambda x: round(((x['count'] / x['count'].sum()) * 100), 0)
    )
)

# sc
bins= [0, 1, 5, 1_000_000]
df_sc_go = (df_onl_go
    .loc[df_onl_go['class_type'] == 'Online Social Club']
    .groupby(['class_type', pd.cut(df_onl_go['count'], bins, right= False)])
    .size()
    .to_frame()
    .rename({0: 'count'}, axis= 'columns')
    .pipe(catchstate, var_name= "df_catched") # catch the current state of df at this point
    .replace(
        to_replace= 0,
        value= num_go_ac - df_catched['count'].sum()
    )
    .assign(
        perc= lambda x: round(((x['count'] / x['count'].sum()) * 100), 0)
    )
)

/tmp/ipykernel_53880/525855635.py:22: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(['class_type', pd.cut(df_onl_go['count'], bins, right= False)])
/tmp/ipykernel_53880/525855635.py:40: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(['class_type', pd.cut(df_onl_go['count'], bins, right= False)])
/tmp/ipykernel_53880/525855635.py:58: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(['class_type', pd

In [38]:
# level 5: basket for online
# ! DLX

# online
df_onl_dlx = (df_att
    .loc[(df_att['student_membership'] == 'Deluxe') &
        (df_att['class_mode'].isin(['Online', 'GOC'])) &
        (df_att['class_type'].isin(['Online Encounter','Online Complementary', 'Online Social Club'])) 
    ]
    .groupby(['class_type', 'student_code'])
    .agg(
        count= ('student_code', 'size')
    )
    .reset_index()
)


# enc
bins = [0, 1, 3, 1_000_000]
df_enc_dlx = (df_onl_dlx
    .loc[df_onl_dlx['class_type'] == 'Online Encounter']
    .groupby(['class_type', pd.cut(df_onl_dlx['count'], bins, right= False)])
    .size()
    .to_frame()
    .rename({0: 'count'}, axis= 'columns')
    .pipe(catchstate, var_name= "df_catched") # catch the current state of df at this point
    .replace(
        to_replace= 0,
        value= num_dlx_ac - df_catched['count'].sum()
    )
    .assign(
        perc= lambda x: round(((x['count'] / x['count'].sum()) * 100), 0)
    )
)

# complementary
bins = [0, 1, 4, 1_000_000]
df_cc_dlx = (df_onl_dlx
    .loc[df_onl_dlx['class_type'] == 'Online Complementary']
    .groupby(['class_type', pd.cut(df_onl_dlx['count'], bins, right= False)])
    .size()
    .to_frame()
    .rename({0: 'count'}, axis= 'columns')
    .pipe(catchstate, var_name= "df_catched") # catch the current state of df at this point
    .replace(
        to_replace= 0,
        value= num_dlx_ac - df_catched['count'].sum()
    )
    .assign(
        perc= lambda x: round(((x['count'] / x['count'].sum()) * 100), 0)
    )
)

# sc
bins = [0, 1, 5, 1_000_000]
df_sc_dlx = (df_onl_dlx
    .loc[df_onl_dlx['class_type'] == 'Online Social Club']
    .groupby(['class_type', pd.cut(df_onl_dlx['count'], bins, right= False)])
    .size()
    .to_frame()
    .rename({0: 'count'}, axis= 'columns')
    .pipe(catchstate, var_name= "df_catched") # catch the current state of df at this point
    .replace(
        to_replace= 0,
        value= num_dlx_ac - df_catched['count'].sum()
    )
    .assign(
        perc= lambda x: round(((x['count'] / x['count'].sum()) * 100), 0)
    )
)

/tmp/ipykernel_53880/1193344190.py:22: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(['class_type', pd.cut(df_onl_dlx['count'], bins, right= False)])
/tmp/ipykernel_53880/1193344190.py:40: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(['class_type', pd.cut(df_onl_dlx['count'], bins, right= False)])
/tmp/ipykernel_53880/1193344190.py:58: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(['class_type

In [39]:
# level 5: basket for incenter
# ! DLX
bin1 = 1
bin2 = [2, 3]
bin3 = list(range(4, 1_000_000))

df_incenter = (
    df_att.loc[
        (df_att["class_mode"] == "Offline") & (df_att["student_membership"] != "VIP")
    ]
    .groupby("student_code")
    .agg(count=("student_code", "size"))
    .reset_index()
)

bin1 = sum(df_incenter["count"] == bin1)
bin2 = sum(df_incenter["count"].isin(bin2))
bin3 = sum(df_incenter["count"].isin(bin3))
bin0 = num_dlx_ac - bin1 - bin2 - bin3

cols = ["0", "1", "2-3", ">=4"]
data = [bin0, bin1, bin2, bin3]
index = ["values"]

df_incenter = pd.DataFrame(data, cols)

df_incenter = df_incenter.rename(columns={0: "count"}).assign(
    perc=lambda df_: round(((df_["count"] / df_["count"].sum()) * 100), 0)
)

In [40]:
#check
df_att.loc[
    (df_att['student_code'] == ', SAMIRAH - jak05.05764.id'),
    ['class_mode']
    ].value_counts()

Series([], Name: count, dtype: int64)

In [41]:
# level 5: basket for vip

df_vip_basket = (df_sess
    .loc[df_sess['class_service'] == 'VIP']
    .groupby('class_type_grouped')
    .agg(
        count= ('class_type_grouped', 'size')
    )
    .assign(
        group= lambda df_: np.select(
            condlist= [
                df_.index.str.contains('VPG', regex= False), 
                df_.index.str.contains('Other', regex= False), 
            ],
            choicelist= ['VPG', 'Other'],
            default= '1:1'
        ),
        perc= lambda df_: round(((df_['count'] / df_.groupby('group')['count'].transform('sum')) * 100), 0)
    )
)

# Output

In [42]:
# level 1 and 2

print(f"Number of members: {num_member}")

print(f"Number of GO: {num_member_go}")
print(f"Number of GO: {perc_go}")

print(f"Number of deluxe: {num_member_dlx}")
print(f"Number of deluxe: {perc_dlx}")

print(f"Number of VIP: {num_member_vip}")
print(f"Number of VIP: {perc_vip}")

print(
    f"""Check = {
    (num_member_go + num_member_dlx + num_member_vip == num_member) &
    (perc_go + perc_dlx + perc_vip >= 99) & 
    (perc_go + perc_dlx + perc_vip <= 101)
    }
"""
)

Number of members: 4016.0
Number of GO: 1284.0
Number of GO: 32.0
Number of deluxe: 2474.0
Number of deluxe: 62.0
Number of VIP: 258.0
Number of VIP: 6.0
Check = True



In [43]:
# level 3

# go
print(f"Number of GO: {num_member_go}")

print(f"\tNumber of active: {num_go_ac}")
print(f"\tPerc of active: {perc_go_ac}")

print(f"\tNumber of inactive: {num_go_in}")
print(f"\tPerc of inactive: {perc_go_in}")

print(
    f"Check = {(num_go_ac + num_go_in == num_member_go) & (perc_go_ac + perc_go_in == 100)}"
)
print("\n")

# dlc
print(f"Number of Deluxe: {num_member_dlx}")

print(f"\tNumber of active: {num_dlx_ac}")
print(f"\tPerc of active: {perc_dlx_ac}")

print(f"\tNumber of inactive: {num_dlx_in}")
print(f"\tPerc of inactive: {perc_dlx_in}")

print(
    f"Check = {(num_dlx_ac + num_dlx_in == num_member_dlx) & (perc_dlx_ac + perc_dlx_in == 100)}"
)
print("\n")

# vip
print(f"Number of VIP: {num_member_vip}")

print(f"\tNumber of active: {num_vip_ac}")
print(f"\tPerc of active: {perc_vip_ac}")

print(f"\tNumber of inactive: {num_vip_in}")
print(f"\tPerc of inactive: {perc_vip_in}")

print(
    f"Check = {(num_vip_ac + num_vip_in == num_member_vip) & (perc_vip_ac + perc_vip_in == 100)}"
)

Number of GO: 1284.0
	Number of active: 556
	Perc of active: 43.0
	Number of inactive: 728.0
	Perc of inactive: 57.0
Check = True


Number of Deluxe: 2474.0
	Number of active: 1184
	Perc of active: 48.0
	Number of inactive: 1290.0
	Perc of inactive: 52.0
Check = True


Number of VIP: 258.0
	Number of active: 175
	Perc of active: 68.0
	Number of inactive: 83.0
	Perc of inactive: 32.0
Check = True


In [44]:
# level 4

# vip one on one vs vpg
print(df_vip)

# num online
print(f"\nnum online: {num_onl}")
print(f"\nCheck = {(num_go_ac + num_dlx_ac == num_onl)}")

                    count
class_type_grouped       
One-on-one            789
VPG                   181

num online: 1740

Check = True


In [45]:
# level 5: basket GO

df_enc_go_clean = df_enc_go.rename_axis(["class_type", "basket"]).reset_index()
df_cc_go_clean = df_cc_go.rename_axis(["class_type", "basket"]).reset_index()
df_sc_go_clean = df_sc_go.rename_axis(["class_type", "basket"]).reset_index()
result = pd.concat([df_enc_go_clean, df_cc_go_clean, df_sc_go_clean])

print(result)

print("\nCheck")
print(result.groupby("class_type").agg({"count": "sum"}))
print(f"should be the same with -> {num_go_ac}")

             class_type        basket  count  perc
0      Online Encounter        [0, 1)    133  24.0
1      Online Encounter        [1, 3)    385  69.0
2      Online Encounter  [3, 1000000)     38   7.0
0  Online Complementary        [0, 1)    229  41.0
1  Online Complementary        [1, 4)    267  48.0
2  Online Complementary  [4, 1000000)     60  11.0
0    Online Social Club        [0, 1)    228  41.0
1    Online Social Club        [1, 5)    222  40.0
2    Online Social Club  [5, 1000000)    106  19.0

Check
                      count
class_type                 
Online Complementary    556
Online Encounter        556
Online Social Club      556
should be the same with -> 556


In [46]:
# level 5: basket DLX

df_enc_dlx_clean = df_enc_dlx.rename_axis(["class_type", "basket"]).reset_index()
df_cc_dlx_clean = df_cc_dlx.rename_axis(["class_type", "basket"]).reset_index()
df_sc_dlx_clean = df_sc_dlx.rename_axis(["class_type", "basket"]).reset_index()
result = pd.concat([df_enc_dlx_clean, df_cc_dlx_clean, df_sc_dlx_clean])

print(result)

print("\nCheck")
print(result.groupby("class_type").agg({"count": "sum"}))
print(f"should be the same with -> {num_dlx_ac}")

             class_type        basket  count  perc
0      Online Encounter        [0, 1)    480  41.0
1      Online Encounter        [1, 3)    621  52.0
2      Online Encounter  [3, 1000000)     83   7.0
0  Online Complementary        [0, 1)    874  74.0
1  Online Complementary        [1, 4)    283  24.0
2  Online Complementary  [4, 1000000)     27   2.0
0    Online Social Club        [0, 1)    810  68.0
1    Online Social Club        [1, 5)    285  24.0
2    Online Social Club  [5, 1000000)     89   8.0

Check
                      count
class_type                 
Online Complementary   1184
Online Encounter       1184
Online Social Club     1184
should be the same with -> 1184


In [47]:
result = df_incenter.transpose()
print(result)

print("\nCheck")

print(result.sum(axis=1))
print(f"should be the same with -> {num_dlx_ac}")

           0      1    2-3    >=4
count  466.0  197.0  182.0  339.0
perc    39.0   17.0   15.0   29.0

Check
count    1184.0
perc      100.0
dtype: float64
should be the same with -> 1184


In [48]:
result = df_vip_basket.transpose()[
    ["Online One-on-one", "One-on-one", "Online VPG", "VPG"]
]
result

class_type_grouped,Online One-on-one,One-on-one,Online VPG,VPG
count,376,413,125,56
group,1:1,1:1,VPG,VPG
perc,48.0,52.0,69.0,31.0


In [49]:
print(result.transpose().groupby("group").sum())

print(f"\nshould be the same with {df_vip}")

      count   perc
group             
1:1     789  100.0
VPG     181  100.0

should be the same with                     count
class_type_grouped       
One-on-one            789
VPG                   181


# Venn Diagram

In [50]:
class_per_student = (
    df_att.sort_values(["student_code", "class_type"])
    .groupby("student_code")
    .agg(class_attended=("class_type", lambda x: ", ".join(x)))
    .reset_index()
)
print(class_per_student.shape)
class_per_student.head()

(1915, 2)


,student_code,class_attended
0,( VIP HO FROM P. CYNTHIA PARAMITHA SIMAMORA) R...,"Online Encounter, Online English Corner, Onlin..."
1,(DLX BSD HO FROM INDRA GUNAWAN CHANDRA) ALETHA...,Online Encounter
2,(DLX BSD) KARISA - 10378,"Online Complementary, Online Complementary, On..."
3,(DLX BSD) NELA - 10452,"Social Club, Social Club, Social Club, Social ..."
4,(DLX BSD) VENDRA - 10386,Online Encounter


In [51]:
enc_only = (class_per_student
    .loc[
        class_per_student['class_attended'].isin(['Encounter', 'Online Encounter'])
    ]
)

non_enc_only = (class_per_student
    .loc[
        ~(class_per_student['class_attended'].str.lower().str.contains('encounter'))
    ]
)

good_member = (class_per_student
    .loc[
        (class_per_student['class_attended'].str.lower().str.contains('encounter')) &\
        (class_per_student['class_attended'].str.lower().str.contains(','))
    ]
)

In [52]:
enc_only_count = enc_only.shape[0]
non_enc_only_count = non_enc_only.shape[0]
good_member_count = good_member.shape[0]

class_active = enc_only_count + non_enc_only_count + good_member_count
perc_class_active = class_active / num_member
dormant = num_member - class_active
perc_dormant = dormant / num_member

perc_enc_only = enc_only_count / num_member
perc_non_enc_only = non_enc_only_count / num_member
perc_good_member = good_member_count / num_member

print(f"without CPT = {config.exclude_cpt}\n")

print(f"num member: {num_member}")
print(
    f"\nclass active: {class_active}, should be the same with -> {num_go_ac + num_dlx_ac + num_vip_ac}"
)
print(f"perc class active : {perc_class_active: .0%}\n")

print(f"non-enc only : {non_enc_only_count} - {perc_non_enc_only: .0%}")
print(f"good member : {good_member_count} - {perc_good_member: .0%}")
print(f"enc only : {enc_only_count} - {perc_enc_only: .0%}")
print(f"dormant : {dormant} - {perc_dormant: .0%}")

without CPT = True

num member: 4016.0

class active: 1915, should be the same with -> 1915
perc class active :  48%

non-enc only : 698 -  17%
good member : 976 -  24%
enc only : 241 -  6%
dormant : 2101.0 -  52%


In [53]:
# from matplotlib_venn import venn2
# from matplotlib import pyplot as plt

# # depict venn diagram
# venn2(subsets = (round(perc_non_enc_only, 2), round(perc_enc_only, 2), round(perc_good_member, 2)), set_labels = ('Non-ENC', 'ENC'))
# plt.show()

# Experiment

In [54]:
# (df_att
#     .loc[
#         df_att['class_type'] == 'Online Social Club'
#     ]
#     .groupby('student_code')
#     .size()
#     .mean()
# )

In [55]:
# # corp member
# df_att.loc[df_att['student_code'].str.lower().str.contains('cpt')]

In [56]:
# (df_att
#     .groupby(['student_membership'])
#     .agg(n= ('student_code', 'nunique'))
# )

In [57]:
# (df_att
#     # .loc[(df_att['student_membership'] == 'GO') &
#     #     (df_att['class_mode'] == 'Online') &
#     #     (df_att['class_type'].isin(['Online Encounter','Online Complementary', 'Online Social Club'])) 
#     # ]
#     # .groupby(['class_type', 'student_code'])
#     # .agg(
#     #     count= ('student_code', 'size')
#     # )
#     # .reset_index()
# )